In [ ]:
import pandas as pd
import logging
log = logging.getLogger()
hdl = logging.StreamHandler()
hdl.setFormatter(logging.Formatter("%(message)s"))
log.addHandler(hdl)
log.setLevel("INFO")

In [ ]:
reader = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', skiprows=2, sep='\t', index_col=0, chunksize=1000)
df_files=pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t', index_col=0)
genes = pd.read_csv("http://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel()

df=pd.DataFrame()

for subdf in reader:
    log.info("new subdf")
    subdf.index = [g[:15] for g in subdf.index] #remove ENSG subversion
    subdf = subdf[subdf.index.isin(genes)]
    subdf = subdf.reindex(index=subdf.index[subdf.index.isin(genes)],
              columns = subdf.columns[subdf.columns.isin(df_files[df_files["SMTS"]=="Brain"].index)]
             )
    df = df.append(subdf)

In [ ]:
df.to_csv("mainTable.csv", index=True, header=True)

In [ ]:
df_files[df_files.index.isin(df.columns)].to_csv("files.dat")

In [ ]:
!rm -r mainTable.csv files.dat